In [2]:
import requests
import logging
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta, date
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from enum import Enum, IntEnum

In [52]:
def get_all_content_infinite_scroll(scroll_pause_time=1):

    page_url = "https://www.pepper.pl/nowe"

    driver = webdriver.Chrome('./chromedriver') 
    driver.get(page_url) 
    time.sleep(3) #time to load a page
    
    screen_height = driver.execute_script("return window.screen.height;")
    i = 1

    #infinite scroll loop
    while True:
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(scroll_pause_time)
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        if (screen_height) * i > scroll_height:
            break

    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    return soup.main.prettify()


print(get_all_content_infinite_scroll(0.2))

/tmp/ipykernel_268/3098802264.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


<main class="flex--expand-v width--all-12 bg--off" id="main">
 <div class="hide--empty" id="nav-layer-top">
 </div>
 <div>
 </div>
 <ul class="messages js-sticky forceLayer" id="messages-list" style="position: fixed; top: 0px; left: 0px; width: 945px;">
 </ul>
 <ul aria-hidden="true" class="messages forceLayer" style="visibility: hidden;">
 </ul>
 <ul aria-hidden="true" class="softMessages-list softMessages-list--top space--mh-2 forceLayer" id="softMessages-list">
 </ul>
 <ul class="notificationTray zIndex--atopFixed notificationTray--top notificationTray--delayed" id="notificationTray-top">
 </ul>
 <div class="flex--expand-v">
  <header class="header aGrid-item--t-0 mute--opacity-0 pointerEvents--none aGrid--sticky width--all-12 forceLayer zIndex--fixed" data-t="nav" role="banner">
   <div class="nav">
    <div class="page-navigation page-center nav-content flex boxAlign-jc--all-sb">
     <div class="navMenu lbox--v-9 space--fromW3-r-2">
      <!-- -->
      <button circle="" class="n

In [49]:
def get_all_article_id():

    page_url = "https://www.pepper.pl/nowe"

    driver = webdriver.Chrome('./chromedriver') 
    driver.get(page_url) 
    time.sleep(3) #time to load a page

    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    articles = soup.find_all('article')

    list_of_article_id = [x["id"] for x in articles] 
    
    return list_of_article_id

print(get_all_article_id())

/tmp/ipykernel_268/139160723.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


['thread_685276', 'thread_685275', 'thread_685271', 'thread_685272', 'thread_685268', 'thread_685269', 'thread_685266', 'thread_685265', 'thread_685263', 'thread_685264', 'thread_685262', 'thread_685261', 'thread_685259', 'thread_685260', 'thread_685258', 'thread_685257', 'thread_685256', 'thread_685255', 'thread_685253', 'thread_685252']


In [2]:
def get_all_article_id_many_subpages(start_page_number=1, end_page_number=2):

    page_number = start_page_number
    list_of_article_id = []

    try:
        while page_number != end_page_number:

            page_url = f"https://www.pepper.pl/nowe?page={page_number}"

            driver = webdriver.Chrome('./chromedriver') 
            driver.get(page_url) 
            time.sleep(0.5) #time to load a page

            page = driver.page_source
            soup = BeautifulSoup(page, 'html.parser')

            articles = soup.find_all('article')

            for i in articles:
                if i["id"] not in list_of_article_id:
                    list_of_article_id.append(i["id"])
            page_number += 1
    except:
        print("Exception occured")

    return list_of_article_id

print(get_all_article_id_many_subpages(1,5))

/tmp/ipykernel_317/2568915977.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


['thread_685389', 'thread_685387', 'thread_685386', 'thread_685385', 'thread_685382', 'thread_685384', 'thread_685383', 'thread_685381', 'thread_685380', 'thread_685378', 'thread_685376', 'thread_684824', 'thread_685368', 'thread_685375', 'thread_685374', 'thread_685373', 'thread_684800', 'thread_685370', 'thread_685341', 'thread_685366', 'thread_685365', 'thread_685338', 'thread_685336', 'thread_685363', 'thread_685361', 'thread_685360', 'thread_685358', 'thread_685356', 'thread_685355', 'thread_685353', 'thread_685352', 'thread_685351', 'thread_685350', 'thread_685349', 'thread_685348', 'thread_685346', 'thread_685345', 'thread_685344', 'thread_685342', 'thread_685339', 'thread_685337', 'thread_685335', 'thread_685334', 'thread_685332', 'thread_685331', 'thread_685329', 'thread_685328', 'thread_685327', 'thread_685326', 'thread_685325', 'thread_685324', 'thread_685241', 'thread_685323', 'thread_685322', 'thread_685319', 'thread_685318', 'thread_685314', 'thread_685317', 'thread_68531

In [14]:
def get_item_name_like(item):

    page_url = "https://www.pepper.pl/nowe"

    driver = webdriver.Chrome('./chromedriver') 
    driver.get(page_url) 
    time.sleep(3) #time to load a page

    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    list_all_items_info = soup.find_all(attrs={'class': "cept-tt thread-link linkPlain thread-title--list js-thread-title"})

    list_all_items_names = [x.get_text() for x in list_all_items_info]
    
    matches = [x for x in list_all_items_names if item.lower() in x.lower()]

    print(matches)


get_item_name_like("Monitor")

/tmp/ipykernel_268/2350172399.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


['Monitor Samsung M8 z kamerką różowy i zielony', 'Monitor Samsung Odyssey ARK 55"']


In [16]:
def get_content_item_tag():

    page_url = "https://www.pepper.pl/nowe"

    driver = webdriver.Chrome('./chromedriver') 
    driver.get(page_url) 
    time.sleep(3)

    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    all_tags_content = soup.find_all(lambda tag:tag.name=="article")
    
    for i in all_tags_content:
        item_name = i.find_all(attrs={'class': "cept-tt thread-link linkPlain thread-title--list js-thread-title"})
        discount_price = i.find_all(attrs={'class': "thread-price text--b cept-tp size--all-l size--fromW3-xl"})
        regular_price = i.find_all(attrs={'class': "mute--text text--lineThrough size--all-l size--fromW3-xl"})
        percentage_discount = i.find_all(attrs={'class': "space--ml-1 size--all-l size--fromW3-xl"})
        item_link = i.find_all('a', href=True, text=True)
        
        #(attrs={'class': "cept-tt thread-link linkPlain thread-title--list js-thread-title"}, href=True)
        

        article_id = i["id"]
        print(item_name[0].get_text())
        try:    
            print(float(discount_price[0].get_text().strip('zł').replace('.','').replace(',','.')))
        except: 
            print("N/A")
        try:
            print(float(regular_price[0].get_text().strip('zł').replace('.','').replace(',','.')))
        except:
            print("N/A")
        try:
            print(float(percentage_discount[0].get_text().strip('%')))
        except:
            print("N/A")
        print(item_link[0]['href'])
        print(article_id.strip('thread_'))


    #print(result)




get_content_item_tag()

/tmp/ipykernel_241/842682866.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


Smartfon POCO X5 PRO Global 8/256GB - czarny i niebieski - 305€
1365.35
1780.35
-23.0
https://www.pepper.pl/promocje/smartfon-poco-x5-pro-global-8256gb-czarny-i-niebieski-305eur-685548
685548
Awokado zielone HASS 1szt. @Netto
1.49
N/A
N/A
https://www.pepper.pl/promocje/awokado-zielone-hass-1szt-at-netto-685547
685547
Smartfon Realme 9 5G 4/128 GB
769.0
N/A
N/A
https://www.pepper.pl/promocje/smartfon-realme-9-5g-4128-gb-685546
685546
Zestawienie klapków kąpielowych Ellesse Filippo (wiele rozmiarów i modeli) @ sportrabat
69.95
119.99
-42.0
https://www.pepper.pl/promocje/zestawienie-klapek-kapielowych-ellesse-filippo-wiele-rozmiarow-i-modeli-at-sportrabat-685545
685545
3 noce dla dwojga w Hotel Magnus Resort ★★★★ (Bystra, Śląskie) - w cenie śniadania i strefa wellness @ Travelist
750.0
945.0
-21.0
https://www.pepper.pl/promocje/3-noce-dla-dwojga-w-hotel-magnus-resort-bystra-slaskie-w-cenie-sniadania-i-strefa-wellness-at-travelist-685544
685544
Miody Pasieki Sadowskich darmowa dostawa InPo

In [ ]:
class Months(Enum):

    sty = '01'
    lut = '02'
    mar = '03'
    kwi = '04'
    maj = '05'
    cze = '06'
    lip = '07'
    sie = '08'
    wrz = '09'
    paz = '10'
    paź = '10'
    lis = '11'
    gru = '12'

    @classmethod
    def to_dict(cls):
        """Returns a dictionary representation of the enum."""
        return {e.name: e.value for e in cls}
    
    @classmethod
    def keys(cls):
        """Returns a list of all the enum keys."""
        return cls._member_names_
    
    @classmethod
    def values(cls):
        """Returns a list of all the enum values."""
        return list(cls._value2member_map_.keys())

def data_conversion(publication_date):

    try:
        if publication_date.endswith(('min', 'g', 's', 'temu')):
            prepared_data = date.today().strftime("%d-%m-%Y")
            return prepared_data
        elif publication_date.startswith(tuple(Months.keys())) and len(publication_date) < 8:      
            if len(publication_date[4:]) == 3:
                day = publication_date[4:6]
            else:
                day = publication_date[4:5].zfill(2)
            month = Months.__members__[publication_date[0:3]].value
            year = str(date.today().year)
            prepared_data = '-'.join([str(day), month, year])
            return prepared_data
        else:
            day = publication_date[4:6]
            month = Months.__members__[publication_date[0:3]].value
            year = publication_date[7:13]
            prepared_data = '-'.join([day, month, year])
            return prepared_data
    except KeyError as e:
        return f'Invalid name of the month: {e}'
    except:
        print('error')



def get_article_data():

    data_pattern = "\d{2}[/.-]\d{2}[/.-]\d{4}"

    page_url = "https://www.pepper.pl/nowe?page=25"

    driver = webdriver.Chrome('./chromedriver') 
    driver.get(page_url) 
    time.sleep(3)

    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    all_tags_content = soup.find_all(lambda tag:tag.name=="article")

    n=1

    for i in all_tags_content:
        publication_date_tag = i.find_all(attrs={'class': "hide--fromW3"})
        item_name = i.find_all(attrs={'class': "cept-tt thread-link linkPlain thread-title--list js-thread-title"})
        print(f"item number {n}")
        n+=1
        print(item_name[0].get_text()) 

        print(publication_date_tag)
        potentially_data_string_1 = publication_date_tag[0].get_text()
        potentially_data_string_2 = publication_date_tag[1].get_text()
        try:
            formatted_data_1 = data_conversion(potentially_data_string_1)

            if bool(re.search(data_pattern, formatted_data_1)):
                print(data_conversion(potentially_data_string_1) + " string 1 success")
            else:
                raise Exception("didnt work")
        except:
            try: 
                formatted_data_2 = data_conversion(potentially_data_string_2)

                if bool(re.search(data_pattern, formatted_data_2)):
                    print(data_conversion(potentially_data_string_2) + " string 2 success")
                else:
                    raise Exception("didnt work second position")
            except:
                potentially_data_string_3 = publication_date_tag[2].get_text()
                print(data_conversion(potentially_data_string_3) + " string 3 success")
 
       

print(get_article_data())

In [7]:
action_type = "/nowe?page="
start_page = 25
end_page = 25
website_url = "https://www.pepper.pl"

class ScrapWebpage:

    def scrap_data(self, website_url, start_page, action_type):

        try:
            url_to_scrap = website_url + action_type + str(start_page)
            driver = webdriver.Chrome('./chromedriver') 
            driver.get(url_to_scrap) 
            time.sleep(3)
            page = driver.page_source
            soup = BeautifulSoup(page, 'html.parser')
            return soup
        except requests.exceptions.ConnectionError as error:
            print(f"ConnectionError occured: {error}. \nTry again later")
        except requests.exceptions.MissingSchema as error:
            print(f"MissingSchema occured: {error}. \nMake sure that protocol indicator is icluded in the website url")
        except requests.exceptions.HTTPError as error:
            print(f"HTTPError occured: {error}. \nMake sure that website url is valid")
        except requests.exceptions.ReadTimeout as error:
            print(f"ReadTimeout occured: {error}. \nTry again later")

    def infinite_scroll():
        pass

    

    

output = ScrapWebpage()
output.scrap_data(website_url, start_page, action_type)

/tmp/ipykernel_20741/4148578175.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


NameError: name 'WebDriverException' is not defined

In [70]:
type = "/nowe?page="
start_page = 25
end_page = 25
website_url = "https://www.pepper.pl"

url_to_scrap = website_url + type + str(start_page)
print(url_to_scrap)

https://www.pepper.pl/nowe?page=25


In [ ]:
class Months(Enum):

    sty = '01'
    lut = '02'
    mar = '03'
    kwi = '04'
    maj = '05'
    cze = '06'
    lip = '07'
    sie = '08'
    wrz = '09'
    paz = '10'
    paź = '10'
    lis = '11'
    gru = '12'

    @classmethod
    def to_dict(cls):
        """Returns a dictionary representation of the enum."""
        return {e.name: e.value for e in cls}
    
    @classmethod
    def keys(cls):
        """Returns a list of all the enum keys."""
        return cls._member_names_
    
    @classmethod
    def values(cls):
        """Returns a list of all the enum values."""
        return list(cls._value2member_map_.keys())

class ItemAddedDataRetrieve(Months):

    def data_format_conversion(self, scraped_publication_date):

        old_dates_data_pattern = "[A-Za-z]+\s\d\d\.\s[0-9]+"

        try:
            if scraped_publication_date.endswith(('min', 'g', 's', 'temu')):
                prepared_data = date.today().strftime("%d-%m-%Y")
                return prepared_data
            elif scraped_publication_date.startswith(tuple(Months.keys())) and len(scraped_publication_date) < 8:      
                if len(scraped_publication_date[4:]) == 3:
                    day = scraped_publication_date[4:6]
                else:
                    day = scraped_publication_date[4:5].zfill(2)
                month = Months.__members__[scraped_publication_date[0:3]].value
                year = str(date.today().year)
                prepared_data = '-'.join([str(day), month, year])
                return prepared_data
            elif bool(re.search(old_dates_data_pattern, scraped_publication_date)):
                day = scraped_publication_date[4:6]
                month = Months.__members__[scraped_publication_date[0:3]].value
                year = scraped_publication_date[8:13]
                prepared_data = '-'.join([day, month, year])
                return prepared_data
        except KeyError as error:
            raise KeyError(f"Invalid name of the month {error}")


    def first_index_position_data_searching(self, html_tag_with_publication_data):

        output_data_pattern = "\d{2}[/.-]\d{2}[/.-]\d{4}"

        string_to_check = html_tag_with_publication_data[0].get_text()

        try:
            formatted_data = self.data_format_conversion(string_to_check)
            if bool(re.search(output_data_pattern, formatted_data)):
                return f"{data_conversion(string_to_check)} + ' string 1 success'"
        except Exception as error:
            print(error)

    def second_index_position_data_searching(self, html_tag_with_publication_data):
        
        output_data_pattern = "\d{2}[/.-]\d{2}[/.-]\d{4}"

        string_to_check = html_tag_with_publication_data[1].get_text()

        try:
            formatted_data = self.data_format_conversion(string_to_check)
            if bool(re.search(output_data_pattern, formatted_data)):
                return f"{data_conversion(string_to_check)} + ' string 2 success'"
        except Exception as error:
            print(error)

    def third_index_position_data_searching(self, html_tag_with_publication_data):
        
        output_data_pattern = "\d{2}[/.-]\d{2}[/.-]\d{4}"

        string_to_check = html_tag_with_publication_data[2].get_text()

        try:
            formatted_data = self.data_format_conversion(string_to_check)
            if bool(re.search(output_data_pattern, formatted_data)):
                return f"{data_conversion(string_to_check)} + ' string 3 success'"
        except Exception as error:
            print(error)

    def scrap_page_and_receive_data



In [61]:
def data_format_conversion(scraped_publication_date):

        old_dates_data_pattern = "[A-Za-z]+\s\d\d\.\s[0-9]+"

        try:
            if scraped_publication_date.endswith(('min', 'g', 's', 'temu')):
                prepared_data = date.today().strftime("%d-%m-%Y")
                return prepared_data
            elif scraped_publication_date.startswith(tuple(Months.keys())) and len(scraped_publication_date) < 8:      
                if len(scraped_publication_date[4:]) == 3:
                    day = scraped_publication_date[4:6]
                else:
                    day = scraped_publication_date[4:5].zfill(2)
                month = Months.__members__[scraped_publication_date[0:3]].value
                year = str(date.today().year)
                prepared_data = '-'.join([str(day), month, year])
                return prepared_data
            elif bool(re.search(old_dates_data_pattern, scraped_publication_date)):
                day = scraped_publication_date[4:6]
                month = Months.__members__[scraped_publication_date[0:3]].value
                year = scraped_publication_date[8:13]
                prepared_data = '-'.join([day, month, year])
                return prepared_data
        except KeyError as error:
            raise KeyError(f"Invalid name of the month {error}")



print(data_format_conversion("u345345435345"))

None
